In [ ]:
!pip install beir
!pip install -U sentence-transformers
!pip install rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 KB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/21

In [ ]:
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader

import logging, pathlib, os, string
from tqdm.notebook import tqdm

from rank_bm25 import BM25Okapi

import numpy as np
import pandas as pd
import re
from datetime import timedelta
import time
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

from sklearn.metrics import ndcg_score

from sentence_transformers import SentenceTransformer

import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on {device}")

Running on cpu


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


We include the following datasets in BEIR:

| Dataset   | Website| BEIR-Name | Domain     | Relevancy| Queries  | Documents | Avg. Docs/Q | Download | 
| -------- | -----| ---------| ----------- | ---------| ---------| --------- | ------| ------------| 
| MSMARCO    | [``Homepage``](https://microsoft.github.io/msmarco/)| ``msmarco`` | Misc.       |  Binary  |  6,980   |  8.84M     |    1.1 | Yes |  
| TREC-COVID |  [``Homepage``](https://ir.nist.gov/covidSubmit/index.html)| ``trec-covid``| Bio-Medical |  3-level|50|  171K| 493.5 | Yes | 
| NFCorpus   | [``Homepage``](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) | ``nfcorpus``  | Bio-Medical |  3-level |  323     |  3.6K     |  38.2 | Yes |
| BioASQ     | [``Homepage``](http://bioasq.org) | ``bioasq``| Bio-Medical |  Binary  |   500    |  14.91M    |  8.05 | No | 
| NQ         | [``Homepage``](https://ai.google.com/research/NaturalQuestions) | ``nq``| Wikipedia   |  Binary  |  3,452   |  2.68M  |  1.2 | Yes | 
| HotpotQA   | [``Homepage``](https://hotpotqa.github.io) | ``hotpotqa``| Wikipedia   |  Binary  |  7,405   |  5.23M  |  2.0 | Yes |
| FiQA-2018  | [``Homepage``](https://sites.google.com/view/fiqa/) | ``fiqa``    | Finance     |  Binary  |  648     |  57K    |  2.6 | Yes | 
| Signal-1M (RT) | [``Homepage``](https://research.signal-ai.com/datasets/signal1m-tweetir.html)| ``signal1m`` | Twitter     |  3-level  |   97   |  2.86M  |  19.6 | No |
| TREC-NEWS  | [``Homepage``](https://trec.nist.gov/data/news2019.html) | ``trec-news``    | News     |  5-level  |   57    |  595K    |  19.6 | No |
| ArguAna    | [``Homepage``](http://argumentation.bplaced.net/arguana/data) | ``arguana`` | Misc.       |  Binary  |  1,406     |  8.67K    |  1.0 | Yes |
| Touche-2020| [``Homepage``](https://webis.de/events/touche-20/shared-task-1.html) | ``webis-touche2020``| Misc.       |  6-level  |  49     |  382K    |  49.2 |  Yes |
| CQADupstack| [``Homepage``](http://nlp.cis.unimelb.edu.au/resources/cqadupstack/) | ``cqadupstack``| StackEx.      |  Binary  |  13,145 |  457K  |  1.4 |  Yes |
| Quora| [``Homepage``](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs) | ``quora``| Quora  | Binary  |  10,000     |  523K    |  1.6 |  Yes | 
| DBPedia | [``Homepage``](https://github.com/iai-group/DBpedia-Entity/) | ``dbpedia-entity``| Wikipedia |  3-level  |  400    |  4.63M    |  38.2 |  Yes | 
| SCIDOCS| [``Homepage``](https://allenai.org/data/scidocs) | ``scidocs``| Scientific |  Binary  |  1,000     |  25K    |  4.9 |  Yes | 
| FEVER| [``Homepage``](http://fever.ai) | ``fever``| Wikipedia     |  Binary  |  6,666     |  5.42M    |  1.2|  Yes | 
| Climate-FEVER| [``Homepage``](http://climatefever.ai) | ``climate-fever``| Wikipedia |  Binary  |  1,535     |  5.42M |  3.0 |  Yes |
| SciFact| [``Homepage``](https://github.com/allenai/scifact) | ``scifact``| Scientific |  Binary  |  300     |  5K    |  1.1 |  Yes |

In [ ]:
k_1 = 10
k = 10

#Load dataset

In [ ]:
def downoad_and_get_data(dataset):
  data_path = f'datasets/{dataset}'
  url = f'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  print(f'Dataset downloaded here: {data_path}')

  train_d, train_q, train_lab_q = GenericDataLoader(data_path).load('train')
  _, test_q, test_lab_q = GenericDataLoader(data_path).load('test')

  return ({'ds': pd.DataFrame.from_dict(train_d, orient='index'), 'qs': pd.DataFrame.from_dict(train_q, orient='index'), 'lq':  pd.DataFrame.from_dict(train_lab_q, orient='index').fillna(0)},
  {'qs': pd.DataFrame.from_dict(test_q, orient='index'), 'lq':  pd.DataFrame.from_dict(test_lab_q, orient='index').fillna(0)})

In [ ]:
dataset = 'scifact'
train_dic, test_dic = downoad_and_get_data(dataset)

Dataset downloaded here: /content/datasets/scifact


  0%|          | 0/5183 [00:00<?, ?it/s]

  0%|          | 0/5183 [00:00<?, ?it/s]

# Queries & Documents Pre-Processing

In [ ]:
def pre_processing_text(text):
  ps = PorterStemmer()
  en_stops = set(stopwords.words('english'))

  text = re.sub(r'#', '', re.sub(r'https?:\/\/.*[\r\n]*', '', text))
  words = word_tokenize(text.translate(str.maketrans('', '', string.punctuation)))
  #return np.array([ps.stem(processed_word.lower()) for processed_word in words if processed_word not in en_stops], dtype=object)
  return [ps.stem(processed_word.lower()) for processed_word in words if processed_word not in en_stops]


def text_pre_processing(train_dic, test_dic):

  train_dic['ds'] = train_dic['ds'].drop(['title'], axis=1)
  train_dic['qs'].columns = ['text']
  test_dic['qs'].columns = ['text']
  
  tqdm.pandas(desc='Train Documents Pre-Processing')
  train_dic['ds']['text'] = train_dic['ds']['text'].progress_map(pre_processing_text)

  tqdm.pandas(desc='Train Queries Pre-Processing')
  train_dic['qs']['text'] =  train_dic['qs']['text'].progress_map(pre_processing_text)

  tqdm.pandas(desc='Test Queries Pre-Processing')
  test_dic['qs']['text'] = test_dic['qs']['text'].progress_map(pre_processing_text)

  # Mettere gli indici a int64
  train_dic['ds'].index.astype('int64')
  train_dic['qs'].index.astype('int64')
  train_dic['lq'].columns = pd.to_numeric(list(train_dic['lq'].columns))
  test_dic['qs'].index.astype('int64')
  test_dic['lq'].columns = pd.to_numeric(list(test_dic['lq'].columns))

  return train_dic, test_dic

In [ ]:
train_dic, test_dic = text_pre_processing(train_dic, test_dic)

Train Documents Pre-Processing:   0%|          | 0/5183 [00:00<?, ?it/s]

Train Queries Pre-Processing:   0%|          | 0/809 [00:00<?, ?it/s]

Test Queries Pre-Processing:   0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
print(train_dic['ds'].index, train_dic['ds'].columns)
print(train_dic['qs'].index, train_dic['qs'].columns)
print(train_dic['lq'].index, train_dic['lq'].columns)

Index(['4983', '5836', '7912', '18670', '19238', '33370', '36474', '54440',
       '70115', '70490',
       ...
       '167944455', '168265642', '195317463', '195680777', '195683603',
       '195689316', '195689757', '196664003', '198133135', '198309074'],
      dtype='object', length=5183) Index(['text'], dtype='object')
Index(['0', '2', '4', '6', '9', '10', '11', '12', '14', '15',
       ...
       '1397', '1398', '1400', '1401', '1402', '1403', '1404', '1405', '1406',
       '1407'],
      dtype='object', length=809) Index(['text'], dtype='object')
Index(['0', '2', '4', '18', '6', '25', '9', '10', '11', '62',
       ...
       '1391', '1392', '1393', '1394', '1397', '1398', '1401', '1405', '1406',
       '1407'],
      dtype='object', length=809) Int64Index([31715818, 13734012, 22942787,  2613775, 44265107, 32587939,
            33409100,   641786, 22080671,  1606628,
            ...
             2000038, 12440953,  2251426, 17717391,  5185871, 38355793,
            10504681,  26178

# Sparse Representation

In [ ]:
bm25 = BM25Okapi(train_dic['ds']['text'].to_list())
sparse_embeddings = {}

def process_bm25(bm25, sub_df):
  actual_embeddings = {}
  for idx, query in sub_df.iterrows():
    actual_embeddings[idx] = bm25.get_scores(query['text'])
  return actual_embeddings

In [ ]:
start_time = time.time()

with ProcessPoolExecutor(max_workers=multiprocessing.cpu_count()) as pool:
  with tqdm(total = train_dic['qs'].shape[0] / multiprocessing.cpu_count(), leave=False, desc='Obtaining Queries - Documents Sparse Embeddings') as progress: # Setting up the progress bar

    for idx_query in range(0, train_dic['qs'].shape[0], multiprocessing.cpu_count()):
      future = pool.submit(process_bm25, bm25, train_dic['qs'].iloc[idx_query:idx_query + multiprocessing.cpu_count()])
      future.add_done_callback(lambda p: progress.update())
      sparse_embeddings.update(future.result())

print(f'--- Sparse Embeddings done in {(timedelta(seconds=(time.time() - start_time)))} (H:M:S:ms) ---')

Obtaining Queries - Documents Sparse Embeddings:   0%|          | 0/404.5 [00:00<?, ?it/s]

--- Sparse Embeddings done in 0:02:15.648068 (H:M:S:ms) ---


In [ ]:
sparse_embeddings_df = pd.DataFrame.from_dict(sparse_embeddings, orient='index')
sparse_embeddings_df.columns = pd.to_numeric(list(train_dic['ds'].index))
sparse_embeddings_df.head()

Int64Index([     4983,      5836,      7912,     18670,     19238,     33370,
                36474,     54440,     70115,     70490,
            ...
            167944455, 168265642, 195317463, 195680777, 195683603, 195689316,
            195689757, 196664003, 198133135, 198309074],
           dtype='int64', length=5183)

## Computing NDCG and MRR

In [ ]:
for col in list(train_dic['ds'].index):
  if col not in list(train_dic['lq'].columns):
    train_dic['lq'][col] = 0

# Non switcha le colonne
train_dic['lq'].columns = sorted(pd.to_numeric(train_dic['lq'].columns))

In [ ]:
#MMR@K and NDCG@K
sum_ndcg = 0
for idx, query_doc_score in sparse_embeddings_df.iterrows():
  sum_ndcg += ndcg_score(train_dic['lq'].loc[idx], query_doc_score.to_numpy())
ndcg = sum_ndcg / query_doc_score.shape[0]

ValueError: ignored

In [ ]:
'''if(False):
  for idx, res in query_doc_scores.items():
    winner = np.argmax(res)
    print(f'Predicted - Best document for query {idx} is the {list(corpus.keys())[winner]}')
    print(f'Ground Truth - Best document for query {idx} is the {list(qrels[str(idx)].keys())}\n')'''

# Dense Representation

In [ ]:
'''
batch_size = 128
dense_values = []
for i in tqdm(range(0, len(df), batch_size)):
  dense_values += model.encode(df.iloc[i:i + batch_size]["text"].tolist()).tolist()
'''

In [ ]:
model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)
documents_embeddings = {}
dense_documents_embedding = {}
dense_queries_embedding = {}

for idx, doc_text in tqdm(train_dic['ds'].iterrows(), total=train_dic['ds'].shape[0], desc='Obtaining Dense Documents Embeddings'):
  #dense_documents_embedding[idx] = model.encode(doc_text['text'])

for idx, doc_text in tqdm(train_dic['ds'].iterrows(), total=train_dic['ds'].shape[0], desc='Obtaining Dense Queries Embeddings'):
  #dense_queries_embedding[idx] = model.encode(doc_text['text'])


KeyboardInterrupt: ignored

## Computing NDCG and MRR